<a href="https://colab.research.google.com/github/xamevou/someJupyterNotebooks/blob/main/KeyAndAddressGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bitcoin

  Created wheel for bitcoin: filename=bitcoin-1.1.42-py3-none-any.whl size=44411 sha256=5ff71aa16d7984e1fdb3af55bfe26cac25de62b43d1e17eebb8b82059191cee0
  Stored in directory: /home/jovyan/.cache/pip/wheels/6c/66/7d/2b4114252aed67e75a79a1235ce8b2701da915acac6f6d7061
Successfully built bitcoin


In [2]:
# This is example 5 of chaper 4 in Antonopoulos' book
from __future__ import print_function
import bitcoin

In [3]:
# Generate a random private key
# Use this string instead of a random_key(), for reproducing example 4.3 in book, in .cpp
# private_key = "038109007313a5807b2eccc082c8c3fbb988a973cacf1a7df9ce725c31b14776"

valid_private_key = False
while not valid_private_key:    
    private_key = bitcoin.random_key()
    decoded_private_key = bitcoin.decode_privkey(private_key, 'hex')
    valid_private_key = 0 < decoded_private_key < bitcoin.N

print("Private Key (hex) is: ", private_key)
print("Private Key (decimal) is: ", decoded_private_key)

Private Key (hex) is:  346b21610be99217f41188f34b0e6f371c3a8e1bb66f81bf6368421a47fbb4af
Private Key (decimal) is:  23709551136259930733025747731486653601255851866688130493082128939776602780847


In [4]:
# Convert private key to WIF format
wif_encoded_private_key = bitcoin.encode_privkey(decoded_private_key, 'wif')
print("Private Key (WIF) is: ", wif_encoded_private_key)

Private Key (WIF) is:  5JDNYLn6hxstVZVG9jA7aweVMeGEaB9JgfiEZW7zHjyEhdpA6J5


In [5]:
# Add suffix "01" to indicate a compressed private key
compressed_private_key = private_key + '01'
print("Private Key Compressed (hex) is: ", compressed_private_key)

Private Key Compressed (hex) is:  346b21610be99217f41188f34b0e6f371c3a8e1bb66f81bf6368421a47fbb4af01


In [6]:
# Generate a WIF format from the compressed private key (WIF-compressed)
wif_compressed_private_key = bitcoin.encode_privkey(
    bitcoin.decode_privkey(compressed_private_key, 'hex'), 'wif')
print("Private Key (WIF-Compressed) is: ", wif_compressed_private_key)

Private Key (WIF-Compressed) is:  Kxyc5LJ4BodcH4PjsX5E4pCUA7UEuRmf45rzrTXqrtfDUHqFmSUt


In [7]:
# Multiply the EC generator point G with the private key to get a public key point
public_key = bitcoin.fast_multiply(bitcoin.G, decoded_private_key)
print("Public Key (x,y) coordinates is:", public_key)

Public Key (x,y) coordinates is: (1560497365668606840469070636085766981200544734263782961900469613093308650998, 26328635033065480749681875195401641024006127304819074748860268092415974275246)


In [8]:
# Encode as hex, prefix 04
hex_encoded_public_key = bitcoin.encode_pubkey(public_key, 'hex')
print("Public Key (hex) is:", hex_encoded_public_key)

Public Key (hex) is: 04037335d044c934815b6a620f18f5dd4df93eee1681e5be69d6b57657a0b8d5f63a357ab609dba941af02487599de819586904bdc9112288ca84f805b568f78ae


In [9]:
# Compress public key, adjust prefix depending on whether y is even or odd
(public_key_x, public_key_y) = public_key
compressed_prefix = '02' if (public_key_y % 2) == 0 else '03'
hex_compressed_public_key = compressed_prefix + (bitcoin.encode(public_key_x, 16).zfill(64))
print("Compressed Public Key (hex) is:", hex_compressed_public_key)

Compressed Public Key (hex) is: 02037335d044c934815b6a620f18f5dd4df93eee1681e5be69d6b57657a0b8d5f6


In [10]:
# Generate bitcoin address from public key
print("Bitcoin Address (b58check) is:", bitcoin.pubkey_to_address(public_key))

Bitcoin Address (b58check) is: 1tP2YK5fWY1MrjvrWhb5dtCnWsTHjrPtP


In [11]:
# Generate compressed bitcoin address from compressed public key
print("Compressed Bitcoin Address (b58check) is:",
      bitcoin.pubkey_to_address(hex_compressed_public_key))

Compressed Bitcoin Address (b58check) is: 197KmS3WhXTMrrbJ1BV5nUUqDGAPn5Gx9e
